# MILO - Cell type composition analysis

Publication [Dann et al, 2021](https://www.nature.com/articles/s41587-021-01033-z)  
Marioni Lab [GitHub](https://github.com/MarioniLab/miloR)  

Theis lab python adaption with [pertpy](https://github.com/theislab/pertpy)  

Run makeNhoods with refinement_scheme="graph" to avoid calcNhoodDistance and set fdr.weighting="graph-only" testNhoods https://github.com/MarioniLab/miloR/issues/99

In [1]:
options(warn=-1)

In [2]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # miloR
            library(miloR), 
            # library(ggbeeswarm), 
            
            # SingleCellExperiment
            library(SingleCellExperiment), 

            # Data 
            library(tidyverse), 
            library(data.table), 

            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(cowplot), 
            
            # Parallel 
            library(BiocParallel), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [3]:
# Configure reticulate 
use_condaenv(condaenv="p.3.10.16-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
py_config()

python:         /nobackup/peer/fdeckert/miniconda3/envs/p.3.10.16-FD20200109SPLENO/bin/python
libpython:      /nobackup/peer/fdeckert/miniconda3/envs/p.3.10.16-FD20200109SPLENO/lib/libpython3.10.so
pythonhome:     /nobackup/peer/fdeckert/miniconda3/envs/p.3.10.16-FD20200109SPLENO:/nobackup/peer/fdeckert/miniconda3/envs/p.3.10.16-FD20200109SPLENO
version:        3.10.16 | packaged by conda-forge | (main, Apr  8 2025, 20:53:32) [GCC 13.3.0]
numpy:          /nobackup/peer/fdeckert/miniconda3/envs/p.3.10.16-FD20200109SPLENO/lib/python3.10/site-packages/numpy
numpy_version:  1.26.4

NOTE: Python version was forced by use_python() function

In [4]:
random_seed <- 42
set.seed(random_seed)

In [5]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [6]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

In [7]:
# Parallel 
BPPARAM <- SerialParam()
register(BPPARAM)

# Import Seurat object

In [8]:
# Import Seurat object 
so <- readRDS("data/scRNAseq/object/pp_0.rds") 

# Subset to BSF samples with replicates

In [9]:
so@meta.data <- droplevels(so@meta.data)

# Convert Seurat to Milo object

In [10]:
# Get components
counts <- GetAssayData(so, assay="RNA", slot="counts")
meta <- so@meta.data
latent <- so@reductions$latent@cell.embeddings
umap <- so@reductions$umap@cell.embeddings

In [11]:
# SingleCellExperiment object 
sce <- SingleCellExperiment(list(counts=counts), colData=meta, reducedDims=list(latent=latent, umap=umap))

In [12]:
# Milo object
milo <- miloR::Milo(sce)

# Run Milo

In [ ]:
cache_k_range <- FALSE
k_range <- seq(10, 100, 10)

In [ ]:
if(!cache_k_range) {
    
    for (k in k_range) {
    
        # Graph 
        milo <- buildGraph(milo, k=k, d=30, reduced.dim="latent")
        milo <- makeNhoods(milo, k=k, d=30, refined=TRUE, reduced_dims="latent", refinement_scheme="graph")
        milo <- countCells(milo, meta.data=data.frame(colData(milo)), sample="sample_group_rep")
        
        saveRDS(milo, paste0("data/scRNAseq/milo/milo_k", as.character(k), ".rds"))
    
    }
    
}

In [ ]:
milo <- lapply(k_range, function(k) {readRDS(paste0("data/scRNAseq/milo/milo_k", as.character(k), ".rds"))})
names(milo) <- k_range

In [ ]:
options(repr.plot.width=30, repr.plot.height=5)

hist_1 <- lapply(milo, function(i) {plotNhoodSizeHist(i) + ggtitle(paste("Hood size\n k", i@.k)) + xlim(0, 300) + theme_global_set(2)})

wrap_plots(hist_1, ncol=10, nrow=1)

## infection NaCl vs CpG 

In [ ]:
milo_cache <- FALSE

In [ ]:
milo_tl <- function(milo, so, k) {
    
    # Design matrix 
    design_df <- so@meta.data[,c ("sample_group_rep", "infection"), drop=FALSE] %>% dplyr::distinct()
    rownames(design_df) <- design_df$sample_group_rep
    
    infection <- as.character(design_df$infection)
    design <- model.matrix(~0+infection, data=design_df)
    colnames(design) <- gsub("infection", "", colnames(design))

    # Run milo
    res <- testNhoods(milo, design, design_df, reduced.dim="latent", model.contrasts="CpG-NaCl", fdr.weighting="graph-overlap", norm.method="TMM") %>% as.data.frame()
    
    # P-value hist
    pvalue_hist_1 <- ggplot(res, aes(PValue)) + geom_histogram(bins=50) + ggtitle(paste("NaCl vs CpG\n k", milo@.k)) + theme_global_set(1)

    # Volcano plot 
    vp_1 <- ggplot(res, aes(logFC, -log10(SpatialFDR))) + geom_point() + geom_hline(yintercept=1) + ggtitle(paste("NaCl vs CpG\n k", milo@.k)) + theme_global_set(1)
    
    # Annotate hoods
    res <- annotateNhoods(milo, res, coldata_col="celltype_low")
    
    # Factor names 
    res$celltype_low <- factor(res$celltype_low, levels=rev(levels(so$celltype_low))) %>% droplevels()
    res <- res[order(res$celltype_low), ]
    
    # Make names for beeswarm 
    res$celltype_low <- make.names(res$celltype_low)
    
    # Beeswarm 
    if(any(res$SpatialFDR<=0.1)) {
        
        dabp_1 <- suppressMessages(plotDAbeeswarm(res, group.by="celltype_low", alpha=0.1) + scale_color_gradient2(low=color$infection["NaCl"], high=color$infection["CpG"]) + xlab("") + ggtitle(paste("NaCl vs CpG\n k", milo@.k)) + theme_global_set(1))
        
    } else {
        
        dabp_1 <- suppressMessages(plotDAbeeswarm(res, group.by="celltype_low", alpha=1) + xlab("") + ggtitle(paste("k", milo@.k)) + theme_global_set(1))
        
    }
    
    
    # Cobine result table
    res_da <- list(res)
    
    # Combine all
    resl <- list(pvalue_hist=pvalue_hist_1, vp=vp_1, dabp=dabp_1, res_da=res_da)
    
    return(resl)
    
    print("Done")
    
}

In [ ]:
if(!milo_cache) {
    
    resl <- lapply(seq_along(k_range), function(k) milo_tl(milo=milo[[k]], so=so, k=k_range[k]))
    saveRDS(resl, "data/scRNAseq/milo/resl.rds")
                  
} else {
    
    resl <- readRDS("data/scRNAseq/milo/resl.rds")
    
}

In [ ]:
options(repr.plot.width=30, repr.plot.height=5)

wrap_plots(sapply(resl, "[", 1), ncol=10, nrow=1)

In [ ]:
options(repr.plot.width=30, repr.plot.height=5)

wrap_plots(sapply(resl, "[", 2), ncol=10, nrow=1)

In [ ]:
options(repr.plot.width=30, repr.plot.height=10)

wrap_plots(sapply(resl, "[", 3), ncol=10, nrow=1)

# Session info

In [ ]:
sessionInfo()